In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
project_dir = 'C:\\Users\\PLDD\\python\\Python\\ML\\Kaggle\\house_price'
raw_path = os.path.join(project_dir,'data','raw')
train_path = os.path.join(raw_path, 'train.csv')
test_path = os.path.join(raw_path, 'test.csv')

 # Create data frame

Combines train and test data frame:

In [184]:
train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)
test_df['SalePrice'] = 0.0
df = train_df.append(test_df)
df.index = list(range(train_df.index.size + test_df.index.size))
# 2916 out of 2919 Utilities are AllPub so i think it should be droped off
df = df.drop('Utilities', axis = 1)

# Proccess outliers

In [185]:
#'LotFrontage', 'LotArea', 'MasVnrArea', 'GrLivArea', 'GarageArea', 'PoolArea', 'SalePrice']
#copydf['LotFrontage'].plot.hist()
#copydf['LotFrontage'].plot.box()
#copydf['LotArea'].plot.hist()
#copydf['LotArea'].plot.box()
#copydf['GrLivArea'].plot.hist()
#copydf['GrLivArea'].plot.box()
#plt.show()
lx = (df['LotFrontage'] > 200) | (df['LotArea'] > 100000) | (df['GrLivArea'] > 4000)
#df = df.drop(df.index[lx], axis = 0)
df.loc[df['LotFrontage'] > 200,'LotFrontage'] = np.nan
df.loc[df['LotArea'] > 100000,'LotArea'] = np.nan
df.loc[df['GrLivArea'] > 4000,'GrLivArea'] = np.nan

# Proccess Nan values

In [186]:
DefSeqCat = ['Alley','BsmtQual', 'BsmtCond', 'BsmtExposure',
       'BsmtFinType1', 'BsmtFinType2', 'FireplaceQu',
       'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PoolQC',
       'Fence', 'MiscFeature']
UndefSeqCat1 = ['MSZoning', 'Utilities', 'Exterior1st', 'Exterior2nd','Electrical', 'KitchenQual',
            'Functional', 'SaleType'] # there are defenetly lost elements 
UndefSeqCat2 = ['MasVnrType'] # i may just not make it clear the property of the column
UndefSeqNum = ['LotFrontage', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 
               'BsmtFullBath', 'BsmtHalfBath', 'GarageYrBlt', 'GarageCars', 'GarageArea']

replace=dict.fromkeys(DefSeqCat, 'NoNe')
df = df.fillna(replace)
#'MasVnrType'
ix = df.index[df['MasVnrType'].isnull()]
df.loc[ix,'MasVnrType'] = 'NoNe'
df.loc[ix,'MasVnrArea'] = 0

NanCol = ['LotFrontage', 'MSZoning', 'Utilities', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF',
       'Electrical', 'BsmtFullBath', 'BsmtHalfBath', 'KitchenQual',
       'Functional', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'SaleType']

Replace NAN values in 'LotFrontage' and 'LotArea' with iterpolated value.

In [187]:
def ReplaceNanValues_LotFrontageArea(df, alables, abins, fromfld, tofld, catclass, targetclass):
    fullclass = catclass + [targetclass]
    df[tofld]=pd.cut(x = df[fromfld].values, bins = abins, right = False, labels = alables)
    validdf = df[fullclass].dropna(axis = 0, how = 'any')
    nandf = df[df[targetclass].isnull()]
    pvt = validdf.pivot_table(values = targetclass, index = catclass, aggfunc = np.mean)
    pvt=pvt[pvt.notnull().all(1)]
    t1 = pvt.loc[[tuple(x) for x in nandf[catclass].values]]
    t1.index = nandf.index
    df.loc[nandf.index,targetclass] = t1[targetclass]
    df = df.drop(tofld, axis = 1)
    return df

In [188]:
copydf = df.copy()
yearlable = ['1906', '1942', '1975', '1991', '2011']
yearbin = [1800,1906,1942,1975, 1991, 2011]

#case 1
catclass = ['Neighborhood', 'LotConfig','LotShape','YearInt']
copydf = ReplaceNanValues_LotFrontageArea(copydf, yearlable, yearbin, 'YearBuilt', 'YearInt', catclass, 'LotFrontage' )
copydf = ReplaceNanValues_LotFrontageArea(copydf, yearlable, yearbin, 'YearBuilt', 'YearInt', catclass, 'LotArea')
print(copydf[copydf['LotFrontage'].isnull()].index.size)

catclass = ['LotConfig','LotShape','YearInt']
#case 2
copydf = ReplaceNanValues_LotFrontageArea(copydf, yearlable, yearbin, 'YearBuilt', 'YearInt', catclass, 'LotFrontage')
copydf = ReplaceNanValues_LotFrontageArea(copydf, yearlable, yearbin, 'YearBuilt', 'YearInt', catclass, 'LotArea')
#print(copydf[copydf['LotFrontage'].isnull()].index.size)

#case 3
yearlable = ['1942', '1991', '2011']
yearbin = [1800, 1942, 1991, 2011]
copydf = ReplaceNanValues_LotFrontageArea(copydf, yearlable, yearbin, 'YearBuilt', 'YearInt', catclass, 'LotFrontage')
copydf = ReplaceNanValues_LotFrontageArea(copydf, yearlable, yearbin, 'YearBuilt', 'YearInt', catclass, 'LotArea')
#print(copydf[copydf['LotFrontage'].isnull()].index.size)

#case 4
yearlable = ['1975', '1991', '2011']
yearbin = [1800, 1975, 1991, 2011]
copydf = ReplaceNanValues_LotFrontageArea(copydf, yearlable, yearbin, 'YearBuilt', 'YearInt', catclass, 'LotFrontage')
copydf = ReplaceNanValues_LotFrontageArea(copydf, yearlable, yearbin, 'YearBuilt', 'YearInt', catclass, 'LotArea')
#print(copydf[copydf['LotFrontage'].isnull()].index.size)

#case 5
yearlable = ['2011']
yearbin = [1800, 2011]
copydf = ReplaceNanValues_LotFrontageArea(copydf, yearlable, yearbin, 'YearBuilt', 'YearInt', catclass, 'LotFrontage')
copydf = ReplaceNanValues_LotFrontageArea(copydf, yearlable, yearbin, 'YearBuilt', 'YearInt', catclass, 'LotArea')
print(copydf[copydf['LotFrontage'].isnull()].index.size)
print(copydf[copydf['LotArea'].isnull()].index.size)
df = copydf

83
0
0


Replace Nan in 'MSZoning'

In [189]:
ind1 = [1915, 2216, 2250] # C - assign by many factors
ind2 = [2904] # RL - by Lot Area, LotFrontage and YearBuilt 
df.loc[ind1,'MSZoning'] = 'C'
df.loc[ind2,'MSZoning'] = 'RL'

Replace Nan in 'Exterior1st' and 'Exterior2nd'

In [190]:
targetclass = ['Exterior1st', 'Exterior2nd']
ind = [2151]
df.loc[ind,targetclass] = 'AsbShng'

Add column 'RangeYrBlt' and remove columns 'GarageYrBlt' and 'YearBilt'

In [214]:
yearbins = [1800, 1895, 1905, 1917, 1927, 1937,1947,1960, 1973, 1989, 1998, 2012]
yearlabels = ['1895', '1905', '1917', '1927', '1937', '1947', '1960', '1973', '1989', '1998', '2012']
df['RangeYrBlt']=pd.cut(x = df['YearBuilt'].values, bins = yearbins, right = False, labels = yearlabels)
df = df.drop(['YearBuilt', 'GarageYrBlt'], axis = 1)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2919 entries, 0 to 2918
Data columns (total 79 columns):
Id               2919 non-null int64
MSSubClass       2919 non-null int64
MSZoning         2919 non-null object
LotFrontage      2919 non-null float64
LotArea          2919 non-null float64
Street           2919 non-null object
Alley            2919 non-null object
LotShape         2919 non-null object
LandContour      2919 non-null object
LotConfig        2919 non-null object
LandSlope        2919 non-null object
Neighborhood     2919 non-null object
Condition1       2919 non-null object
Condition2       2919 non-null object
BldgType         2919 non-null object
HouseStyle       2919 non-null object
OverallQual      2919 non-null int64
OverallCond      2919 non-null int64
YearRemodAdd     2919 non-null int64
RoofStyle        2919 non-null object
RoofMatl         2919 non-null object
Exterior1st      2919 non-null object
Exterior2nd      2919 non-null object
MasVnrType       2919 

In [211]:
pvt = df.pivot_table(values = 'LotFrontage', index = ['YearBuilt'], aggfunc = np.size)
#pvt.loc[pvt.index > 1950]
#pvt = pvt.loc[pvt[('YearBuilt')] > 1927]
#pvt.sort_index()

In [131]:

targetclass = ['MasVnrType']
alables = ['1906', '1942', '1975', '1991', '2011']
bins = [1800,1906,1942,1975, 1991, 2011]
catclass = [ 'BldgType', 'HouseStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd','Foundation','Neighborhood','MasVnrArea' ]
fullclass = catclass + targetclass
df['YearInt']=pd.cut(x = df['YearBuilt'].values, bins = bins, right = False, labels = alables)
#fullclass.append('YearBuilt')
#fullclass.append()
validdf = df[fullclass].dropna(axis = 0, how = 'any')
nandf = df[df[targetclass].isnull().any(axis = 1)]
nandf[fullclass]

,BldgType,HouseStyle,RoofMatl,Exterior1st,Exterior2nd,Foundation,Neighborhood,MasVnrArea,MasVnrType
234,1Fam,2Story,CompShg,VinylSd,VinylSd,PConc,Gilbert,NaN,NaN
529,1Fam,1Story,CompShg,Wd Sdng,Stone,PConc,Crawfor,NaN,NaN
650,1Fam,2Story,CompShg,CemntBd,CmentBd,PConc,Somerst,NaN,NaN
936,1Fam,1Story,CompShg,VinylSd,VinylSd,PConc,SawyerW,NaN,NaN
973,1Fam,1Story,CompShg,CemntBd,CmentBd,PConc,Somerst,NaN,NaN
977,TwnhsE,1Story,CompShg,VinylSd,VinylSd,PConc,Somerst,NaN,NaN
1243,1Fam,1Story,CompShg,VinylSd,VinylSd,PConc,NridgHt,NaN,NaN
1278,1Fam,2Story,CompShg,VinylSd,VinylSd,PConc,CollgCr,NaN,NaN
1691,1Fam,2Story,CompShg,VinylSd,VinylSd,PConc,Gilbert,NaN,NaN
1706,1Fam,1Story,CompShg,VinylSd,VinylSd,PConc,Somerst,NaN,NaN


In [134]:
pvt = validdf.pivot_table(values = targetclass, index = fullclass, aggfunc = np.size)
#pvt = validdf.pivot_table(values = targetclass, index = ['YearBuilt', , aggfunc = np.size)
#pvt=pvt[pvt.notnull().all(1)]
pvt.loc[('1Fam','1Story', 'CompShg', 'VinylSd', 'VinylSd')]

    #t1 = pvt.loc[[tuple(x) for x in nandf[catclass].values]]
    #t1.index = nandf.index
    #df.loc[nandf.index,targetclass] = t1[targetclass]
    

Foundation  Neighborhood  MasVnrArea  MasVnrType
BrkTil      BrkSide       0.0         None            9
            IDOTRR        0.0         None           18
            OldTown       0.0         None           54
CBlock      BrkSide       0.0         None           36
            CollgCr       0.0         None           36
                          90.0        BrkFace         9
            Edwards       0.0         None           63
                          20.0        Stone           9
            Gilbert       0.0         None            9
            IDOTRR        0.0         None           18
            Mitchel       0.0         None           45
            NAmes         0.0         None          162
                          56.0        BrkFace         9
                          136.0       BrkFace         9
                          172.0       BrkFace         9
                          244.0       BrkFace         9
                          300.0       BrkFace         9

In [73]:

tt[['Neighborhood','YearBuilt', 'LandContour','LandSlope','MSSubClass', 'LotArea','LotFrontage']]

,Neighborhood,YearBuilt,LandContour,LandSlope,MSSubClass,LotArea,LotFrontage
1915,IDOTRR,1910,Lvl,Gtl,30,21780,109.0
1945,Gilbert,1952,Bnk,Gtl,20,31220,195.0


In [13]:
validdf = df#[df['Utilities']]
yy = validdf.pivot_table(values =['LotFrontage', 'LotArea'], index = ['Utilities'], aggfunc = [np.size, np.mean, np.std])
yy

size                      mean                      std  \
          LotArea LotFrontage       LotArea LotFrontage      LotArea   
Utilities                                                              
AllPub       2906      2906.0   9893.742946   69.537952  5131.950539   
NoSeWa          1         1.0  14375.000000   55.407895          NaN   

                       
          LotFrontage  
Utilities              
AllPub      21.605395  
NoSeWa            NaN

In [19]:
#['AllPub', 'NoSeWa', nan]
df[df['Utilities'] == 'AllPub']

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,YearInt
0,1,60,RL,65.000000,8450,Pave,NoNe,Reg,Lvl,AllPub,...,NoNe,NoNe,NoNe,0,2,2008,WD,Normal,208500.0,2011
1,2,20,RL,80.000000,9600,Pave,NoNe,Reg,Lvl,AllPub,...,NoNe,NoNe,NoNe,0,5,2007,WD,Normal,181500.0,2011
2,3,60,RL,68.000000,11250,Pave,NoNe,IR1,Lvl,AllPub,...,NoNe,NoNe,NoNe,0,9,2008,WD,Normal,223500.0,2011
3,4,70,RL,60.000000,9550,Pave,NoNe,IR1,Lvl,AllPub,...,NoNe,NoNe,NoNe,0,2,2006,WD,Abnorml,140000.0,2011
4,5,60,RL,84.000000,14260,Pave,NoNe,IR1,Lvl,AllPub,...,NoNe,NoNe,NoNe,0,12,2008,WD,Normal,250000.0,2011
5,6,50,RL,85.000000,14115,Pave,NoNe,IR1,Lvl,AllPub,...,NoNe,MnPrv,Shed,700,10,2009,WD,Normal,143000.0,2011
6,7,20,RL,75.000000,10084,Pave,NoNe,Reg,Lvl,AllPub,...,NoNe,NoNe,NoNe,0,8,2007,WD,Normal,307000.0,2011
7,8,60,RL,91.666667,10382,Pave,NoNe,IR1,Lvl,AllPub,...,NoNe,NoNe,Shed,350,11,2009,WD,Normal,200000.0,2011
8,9,50,RM,51.000000,6120,Pave,NoNe,Reg,Lvl,AllPub,...,NoNe,NoNe,NoNe,0,4,2008,WD,Abnorml,129900.0,2011
9,10,190,RL,50.000000,7420,Pave,NoNe,Reg,Lvl,AllPub,...,NoNe,NoNe,NoNe,0,1,2008,WD,Normal,118000.0,2011
